# PROGRES - Mini-Projet 2 
# API Web 

Fabien Mathieu - fabien.mathieu@normalesup.org

Sébastien Tixeuil - Sebastien.Tixeuil@lip6.fr

## Etudiants:

- Thierry Ung

- Jack Thay

Le but de ce mini-projet est d'utiliser (entre autres) la bibliothèque Python bottle, pour 
proposer d’une part une API spécifique au site http://dblp.uni-trier.de/ qui 
regroupe l'ensemble des publications scientifiques en informatique, et d’autre part un site web 
qui permet d’utiliser l’API précédente. 

Le site dblp propose l'ensemble des publications sous la forme d'un fichier Xml. Il faut donc 
télécharger ce fichier et utiliser les données qu'il contient afin de créer votre API. Dans la suite, 
on appelle publication, un élément de type `article`, `inproceedings`, `proceedings`, 
`book`, `incollection`, `phdthesis`, ou `mastersthesis`.

Le fichier Xml se trouve à l'adresse http://dblp.uni-trier.de/xml/. Il doit être analysé 
pour récupérer chaque publication (quel que soit son type). L’API demandée porte seulement 
sur les champs `author`, `title`, `year`, `journal`, et `booktitle` (les autres champs 
peuvent donc être ignorés). On peut remarquer que pour une publication, soit le champ 
`journal`, soit le champ `booktitle` est défini (`booktitle` correspond au nom de la 
conférence dans laquelle est publié l’article, `journal` correspond à la revue scientifique dans 
laquelle est publié l’article).

Note: En raison de la quantité limitée de mémoire RAM disponible sur certains ordinateurs, il 
est possible de ne considérer que les publications des 5 dernières années (voire moins, mais 
ce paramètre doit pouvoir être modifié dans le code source).

**RAPPEL :** stackoverflow est votre ami... **Mais** si vous utilisez quelque chose que vous trouvez sur Internet, vous devez citer votre source **ET** ajouter des explications. Ne copiez pas des blocs de code entiers sans comprendre, sans expliquer, sans citer.

# Exercice 1. Mise à disposition d’une API Web 

Réaliser en Python et en utilisant la bibliothèque bottle un serveur Web qui implémente l’API 
Web suivante :
- `/publications/{id}` : avec `id` l'identifiant d'une publication (à vous de choisir quels 
sont les identifiants), qui retourne la publication correspondante.
- `/publications` : qui retourne par défaut les 100 premières publications. La valeur 100 
peut être modifiée au moyen d’un paramètre d'url `limit`).
- `/authors/{name}` : avec `name` le nom d'un auteur, qui retourne des informations 
concernant un auteur : nombres de publications dont il est co-auteur, nombre de co-auteurs.
- `/authors/{name}/publications` : avec `name` le nom d'un auteur, qui liste les 
publications d'un auteur.
- `/authors/{name}/coauthors` : avec `name` le nom d'un auteur, qui liste les co-auteurs 
d'un auteur.
- `/search/authors/{searchString}` : avec `searchString` une chaine de caractères 
permettant de chercher un auteur. Cette route retourne la liste des auteurs dont le nom 
contient `searchString` (par exemple, /search/authors/w retourne la liste de tous les 
auteurs dont le nom contient un `w` ou un `W`). 
- `/search/publications/{searchString}`: avec `searchString` une chaine de 
caractères et qui retourne la liste des publications dont le titre contient `searchString`. La 
route accepte un paramètre d'url `filter` de la forme `key1:value1,key2:value2,...` 
afin d'affiner la recherche aux publications dont la clef `keyi` contient `valuei`. Ainsi, la 
recherche `/search/publications/robots?filter=author:Jean,journal:acm` 
doit retourner la liste des publications dont le titre contient `robots`, dont l'auteur contient 
`Jean` et publiées dans un journal contenant `acm`.
- `/authors/{name_origin}/distance/{name_destination}` : avec `name_origin` 
et `name_destination` deux noms d’auteurs, qui retourne la distance de collaboration 
entre les deux auteurs nommés. Cette distance est définie comme la longueur du plus petit 
chemin `(name_origin, auth1, auth2, ..., authX, name_destination)`, où 
`name_origine` et `auth1` sont co-auteurs, `auth1` et `auth2` sont co-auteurs, ... et 
`authX` et `name_destination` sont co-auteurs. En particulier deux co-auteurs sont à 
distance 1. En plus de retourner la distance, la réponse doit contenir un plus court chemin 
entre les deux auteurs.

In [10]:
%%writefile run_test.py
'''
Created on November 20, 2022

@author: Thierry Ung, Jack Thay

Exercice 1 from MP2
'''
from bottle import *
from re import *
import xml.etree.ElementTree as ET
from lxml import etree as ET
from xml.etree import cElementTree as CET # Element Tree written in C


file_name = "dblp.xml"
p = ET.XMLParser(recover=True)
root = CET.parse(file_name, parser=p).getroot()

@route('/publications/<id>', method="GET")
def publications_id(id):
    compteur = 0
    info = ''
    for i in root.iter('title'):
        if (id == i.text):
            for j in root[compteur]:
                info += j.tag + ': ' + j.text + "<br/>"
            return info
        compteur += 1
    abort(404, "Not found: '/publications/" + id + "'")


run(host='localhost', port=8080, debug = True)


Overwriting run_test.py


In [11]:
!wt python run_test.py

'wt' is not recognized as an internal or external command,
operable program or batch file.


In [5]:
%%writefile test_file.py


from requests import *
from json import *
import pytest


server_ip = "localhost"
server_port = 8080


def test_publications_id():
    r = get(f"http://{server_ip}:{server_port}/publications/Spectre%20Attacks:%20Exploiting%20Speculative%20Execution.")
    assert loads(r.text) == ("author: Paul Kocher<br>author: Daniel Genkin<br>author: Daniel Gruss<br>author: Werner Haas 0004<br>author: Mike Hamburg<br>author: Moritz Lipp<br>author: Stefan Mangard<br>author: Thomas Prescher 0002<br>author: Michael Schwarz 0001<br>author: Yuval Yarom<br>title: Spectre Attacks: Exploiting Speculative Execution.<br>journal: meltdownattack.com<br>year: 2018<br>ee: https://spectreattack.com/spectre.pdf<br>")


Overwriting test_file.py


In [6]:
!python -m pytest test_file.py

============================= test session starts =============================
platform win32 -- Python 3.7.11, pytest-7.1.2, pluggy-1.0.0
rootdir: D:\University\M1 RES\PROGRES\PROGRES_22-23_MP2-main
plugins: anyio-3.5.0
collected 1 item

test_file.py F                                                           [100%]

================================== FAILURES ===================================
____________________________ test_publications_id _____________________________

self = <urllib3.connection.HTTPConnection object at 0x000002E8A5D6AD88>

    def _new_conn(self):
        """Establish a socket connection and set nodelay settings on it.
    
        :return: New socket connection.
        """
        extra_kw = {}
        if self.source_address:
            extra_kw["source_address"] = self.source_address
    
        if self.socket_options:
            extra_kw["socket_options"] = self.socket_options
    
        try:
            conn = connection.create_connection(
>         

In [ ]:
%%writefile run_api_web.py

from bottle import *
from re import *
import xml.etree.ElementTree as ET
import xml.dom.pulldom as PD # For opening large XML files
from lxml import etree as ET

file_name = "dblp.xml"
p = ET.XMLParser(recover=True)
root = PD.parse(file_name)
dict_list = []

@route('/publications/<id>', method="GET")
def publications_id(id):
    compteur = 0
    info = ''
    for i in root.iter('title'):
        if (id == i.text):
            for j in root[compteur]:
                info += j.tag + ': ' + j.text + "<br/>"
            return info
        compteur += 1
    abort(404, "Not found: '/publications/" + id + "'")


run(host='localhost', port=2100, debug = True)


In [ ]:
!wt python run_api_web.py

L’API ainsi développée doit présenter les caractéristiques suivantes :
- Toutes les erreurs doivent avoir le même format.
- Chaque route doit être documentée, avec le format de retour, les erreurs possibles et une 
explications des paramètres.
- Chaque route qui retourne une liste, doit retourner au maximum 100 éléments et accepter 
des paramètre d'URL `start` et `count` qui permettent d'afficher `count` éléments, à partir 
du `start`-ième élément. Par exemple: `/search/authors/*` doit retourner les 100 
premiers auteurs, `/search/authors/*?start=100` affiche les 100 suivants, et `/
search/authors/*?start=200&count=2` affiche les 2 éléments suivants.
- Pour chaque route qui retourne une liste, les éléments retournés doivent pouvoir être triés 
par rapport à un champ donné dans un paramètre d'URL `order`. Par exemple: `/search/
publications/*?order=journal` affiche les 100 premières publications triées dans 
l'ordre alphabétique du nom du journal dans lequel elles apparaissent.

# Exercice 2. Test unitaire d’une API Web 

Réaliser en Python et en utilisant au choix la bibliothèque unittest ou la bibliothèque pytest un programme qui teste le bon 
fonctionnement de l'API Web développée à l'exercice 1.

# Exercice 3. Site Web d’utilisation d’une API Web 

Réaliser en Python et en utilisant la bibliothèque bottle un serveur Web qui utilise l’API Web 
développée à l’exercice 1 pour proposer à l’utilisateur une interface Web graphique qui lui 
permette d’obtenir en entrant les informations pertinentes dans un formulaire Web : 
- la liste complète des publications et la liste complète des coauteurs d’un auteur, possiblement 
triées alphabétiquement. Cet auteur peut au préalable être recherché via une sous-séquence 
de caractères apparaissant dans son nom. 
- La distance entre deux auteurs. Ces auteurs peuvent au préalable être recherchés chacun 
via une sous-séquence de caractères apparaissant dans leur nom.